In [1]:
import pprint as pp
import requests 
import json
from datetime import date
import time
import csv
import datetime
import pandas as pd
import newspaper
from newspaper import Article
from time import mktime
from datetime import datetime, timedelta
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize.regexp import WhitespaceTokenizer
import numpy as np
import matplotlib.pyplot as plt
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from textblob import TextBlob
import praw

[nltk_data] Downloading package punkt to /Users/hsaul/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/hsaul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data_list=  all_articles['articles']
key_words_100=[]
for data in all_articles['articles']:
    art = Article(url=data['url'])
    art.download()
    art.parse()
    art.nlp()
    key_words_100.append(art.keywords)